<a href="https://colab.research.google.com/github/Ola-Crrypt/Ola-Crrypt/blob/main/My_Crypto_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim transformers ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_versio

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.metrics import precision_score, recall_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch
import string
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch
import string
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')


from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch
import string
import ipywidgets as widgets
from IPython.display import display, clear_output


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
# Function to read data from Excel file
def read_excel_data(file_path, sheet_name='sheet2'):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        df = df[['App name', 'review_text', 'review_rating']]
        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return pd.DataFrame()

# Load data (replace with actual file paths)
file1 = '/content/Coinmarketcap review.xlsx'
file2 = '/content/OKX Review.xlsx'
file3 = '/content/crypto.com 1k review.xlsx'

df1 = read_excel_data(file1)
df2 = read_excel_data(file2)
df3 = read_excel_data(file3)

In [3]:
# Combine dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)
df

,App name,review_text,review_rating
0,Coin,I love the fact that I get my info on time here,4
1,Coin,its good,5
2,Coin,very fanatic,5
3,Coin,thank you,5
4,Coin,dope,5
...,...,...,...
2995,Mona,Tried adding my bank to withdraw for over 2 we...,1
2996,Mona,Update- would give 0 stars if possible! I was ...,1
2997,Mona,very slow and limit order doesn't work.,1
2998,Mona,"Annoying set of security measures, seem to be ...",2


In [4]:
# -------------------------------
# 2. Preprocessing
# -------------------------------
def preprocess_text(text):
    text = str(text)
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english') + list(string.punctuation))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens if tokens else ['']

# -------------------------------
# 3. Train/Test Split
# -------------------------------
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Aggregate training reviews by app (profiles)
app_profiles_train = train_df.groupby('App name').agg({
    'review_text': lambda x: ' '.join(x),
    'review_rating': 'mean'
}).reset_index()


### 4. Recommendation Functions

In [5]:
# TF-IDF recommender
def tfidf_recommend_single(user_text, top_n=3):
    vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(app_profiles_train['review_text'])
    user_vec = vectorizer.transform([user_text])
    cosine_sim = cosine_similarity(user_vec, tfidf_matrix).flatten()
    sim_scores = cosine_sim.argsort()[::-1][:top_n]
    return app_profiles_train['App name'].iloc[sim_scores].tolist()

# Word2Vec recommender
def word2vec_recommend_single(user_text, top_n=3):
    tokenized_reviews = [preprocess_text(t) for t in app_profiles_train['review_text']]
    w2v_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

    def get_doc_embedding(tokens):
        vectors = [w2v_model.wv[w] for w in tokens if w in w2v_model.wv]
        return np.mean(vectors, axis=0) if vectors else np.zeros(100)

    app_embeddings = np.array([get_doc_embedding(tokens) for tokens in tokenized_reviews])
    user_tokens = preprocess_text(user_text)
    user_embedding = get_doc_embedding(user_tokens).reshape(1, -1)

    cosine_sim = cosine_similarity(user_embedding, app_embeddings).flatten()
    sim_scores = cosine_sim.argsort()[::-1][:top_n]
    return app_profiles_train['App name'].iloc[sim_scores].tolist()

# BERT recommender
def bert_recommend_single(user_text, top_n=3):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

    app_embeddings = np.array([get_bert_embedding(t) for t in app_profiles_train['review_text']])
    user_embedding = get_bert_embedding(user_text).reshape(1, -1)

    cosine_sim = cosine_similarity(user_embedding, app_embeddings).flatten()
    sim_scores = cosine_sim.argsort()[::-1][:top_n]
    return app_profiles_train['App name'].iloc[sim_scores].tolist()


## 5. Evaluation

In [6]:
def evaluate_method(test_df, method_func, k=3):
    precisions, recalls = [], []

    for _, row in test_df.iterrows():
        true_app = row['App name']
        user_text = row['review_text']

        recommendations = method_func(user_text, top_n=k)

        if not recommendations:
            continue

        # Precision@k: correct if true app is in top-k
        precision = 1 if true_app in recommendations else 0
        # Recall@k: since each review belongs to only one true app, same as precision here
        recall = precision
        precisions.append(precision)
        recalls.append(recall)

    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall + 1e-9)

    return avg_precision, avg_recall, f1

### 6. Run & Compare Models

In [ ]:
results = {}

for name, func in {
    "TF-IDF": tfidf_recommend_single,
    "Word2Vec": word2vec_recommend_single,
    "BERT": bert_recommend_single
}.items():
    p, r, f1 = evaluate_method(test_df, func, k=3)
    results[name] = {"Precision@3": p, "Recall@3": r, "F1@3": f1}
    print(f"{name} → Precision@3: {p:.2f}, Recall@3: {r:.2f}, F1@3: {f1:.2f}")

# -------------------------------
# 7. Plot comparison
# -------------------------------
metrics = ["Precision@3", "Recall@3", "F1@3"]
x = np.arange(len(metrics))
width = 0.25

fig, ax = plt.subplots(figsize=(8,5))
for i, (method, vals) in enumerate(results.items()):
    ax.bar(x + i*width, [vals[m] for m in metrics], width, label=method)

ax.set_xticks(x + width)
ax.set_xticklabels(metrics)
ax.set_ylim(0,1)
ax.set_title("Recommendation Performance Comparison")
ax.legend()
plt.show()

TF-IDF → Precision@3: 1.00, Recall@3: 1.00, F1@3: 1.00
Word2Vec → Precision@3: 1.00, Recall@3: 1.00, F1@3: 1.00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

## Demo

In [7]:
# Aggregate reviews by app
app_profiles = df.groupby('App name').agg({
    'review_text': lambda x: ' '.join(x.astype(str)),
    'review_rating': 'mean'
}).reset_index()

# -------------------------------
# 3. Preprocessing
# -------------------------------
def preprocess_text(text):
    text = str(text).lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english') + list(string.punctuation))
    tokens = [t for t in tokens if t not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens if tokens else [""]

# -------------------------------
# 4. Recommendation methods
# -------------------------------

# TF-IDF
def tfidf_from_text(user_text, top_n=3):
    vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(app_profiles['review_text'])
    user_vec = vectorizer.transform([user_text])
    cosine_sim = cosine_similarity(user_vec, tfidf_matrix).flatten()
    sim_scores = cosine_sim.argsort()[::-1][:top_n]
    return app_profiles.iloc[sim_scores][['App name', 'review_rating']]

# Word2Vec
def word2vec_from_text(user_text, top_n=3):
    tokenized_reviews = [preprocess_text(t) for t in app_profiles['review_text']]
    w2v_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

    def get_embedding(tokens):
        vectors = [w2v_model.wv[w] for w in tokens if w in w2v_model.wv]
        return np.mean(vectors, axis=0) if vectors else np.zeros(100)

    app_embeddings = np.array([get_embedding(tokens) for tokens in tokenized_reviews])
    user_tokens = preprocess_text(user_text)
    user_embedding = get_embedding(user_tokens).reshape(1, -1)

    cosine_sim = cosine_similarity(user_embedding, app_embeddings).flatten()
    sim_scores = cosine_sim.argsort()[::-1][:top_n]
    return app_profiles.iloc[sim_scores][['App name', 'review_rating']]

# BERT
def bert_from_text(user_text, top_n=3):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

    app_embeddings = np.array([get_bert_embedding(t) for t in app_profiles['review_text']])
    user_embedding = get_bert_embedding(user_text).reshape(1, -1)

    cosine_sim = cosine_similarity(user_embedding, app_embeddings).flatten()
    sim_scores = cosine_sim.argsort()[::-1][:top_n]
    return app_profiles.iloc[sim_scores][['App name', 'review_rating']]

# Map methods
text_methods = {
    'TF-IDF': tfidf_from_text,
    'Word2Vec': word2vec_from_text,
    'BERT': bert_from_text
}

# -------------------------------
# 5. Interactive UI with ipywidgets
# -------------------------------
text_input = widgets.Textarea(
    value="",
    placeholder="Type your scenario or preference here...",
    description="Your Text:",
    layout=widgets.Layout(width="70%", height="100px"),
    style={'description_width': 'initial'}
)

method_dropdown = widgets.Dropdown(
    options=list(text_methods.keys()),
    description='Method:',
    layout=widgets.Layout(width="50%"),
    style={'description_width': 'initial'}
)

recommend_button = widgets.Button(
    description="Get Best Apps",
    button_style='success',
    layout=widgets.Layout(width="50%")
)

output = widgets.Output()

def on_text_recommend(b):
    clear_output(wait=True)
    display(text_input, method_dropdown, recommend_button, output)

    user_text = text_input.value.strip()
    if not user_text:
        with output:
            clear_output()
            print("⚠️ Please enter some text.")
        return

    method_name = method_dropdown.value
    method_func = text_methods[method_name]

    recommendations = method_func(user_text, top_n=3)

    with output:
        clear_output()
        print(f"User text: {user_text}\n")
        print(f"Recommendation Method: {method_name}")
        print("✅ Best Recommended Apps:")
        for _, row in recommendations.iterrows():
            print(f" - {row['App name']} (Avg. Rating: {row['review_rating']:.2f})")

recommend_button.on_click(on_text_recommend)
display(text_input, method_dropdown, recommend_button, output)

Textarea(value='Instant buy and sell app', description='Your Text:', layout=Layout(height='100px', width='70%'…

Dropdown(description='Method:', layout=Layout(width='50%'), options=('TF-IDF', 'Word2Vec', 'BERT'), style=Desc…

Button(button_style='success', description='Get Best Apps', layout=Layout(width='50%'), style=ButtonStyle())

Output()